## Simple GenAI app using Langchain

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'

In [3]:
from langchain_community.document_loaders import WebBaseLoader

data = WebBaseLoader("https://arxiv.org/abs/1706.03762").load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

docs = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 50).split_documents(data)

In [22]:
from langchain_community.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [23]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [29]:
query = "Which models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration?"

res = db.similarity_search_with_score(query)
for i in res:
    print(i[0].page_content, " --> ", i[1])

are based on complex recurrent or convolutional neural networks in an encoder-decoder  -->  0.13673447
Abstract:The dominant sequence transduction models are based on complex recurrent or convolutional  -->  0.33535013
neural networks in an encoder-decoder configuration. The best performing models also connect the  -->  0.36871958
the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions  -->  0.38919857


## Retrievel Chain

In [31]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'gpt-4o')

In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
        Provide me answers in few words based on the following context:
        <context>
        {context}
        </context>
    """
)

In [39]:
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Provide me answers in few words based on the following context:\n        <context>\n        {context}\n        </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000021AFE199A20>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000021AFE19BB20>, root_client=<openai.OpenAI object at 0x0000021AFE05E6B0>, root_async_client=<openai.AsyncOpenAI object at 0x0000021AFE199A80>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={},

In [40]:
from langchain_core.documents import Document

document_chain.invoke({
    "input" : query,
    "context" : [Document(page_content = "The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism.")]
})

'1. Dominant sequence transduction models: Recurrent or convolutional neural networks\n2. Model configuration: Encoder-decoder\n3. Best performing models feature: Attention mechanism'

However, we want the documents to first come from the retriever. So we don't have to select context manually.
we can use the retriever to dynamically select the most relevant context and pass those in for a given question.

In [41]:
retriever = db.as_retriever()

In [43]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021AC0CD5ED0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Provide me answers in few words based on the following context:\n        <context>\n        {context}\n        </context>\n    '), addition

In [45]:
res = retrieval_chain.invoke({"input" : query})
res['answer']

'1. Dominant models are based on?\n   - Recurrent or convolutional neural networks.\n\n2. What configuration do these models use?\n   - Encoder-decoder configuration.\n\n3. Which model dispenses with recurrence and convolutions?\n   - The Transformer.\n\n4. What mechanism does the Transformer rely on?\n   - Attention mechanisms.'

In [48]:
print(query)
print(res['answer'])

Which models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration?
1. Dominant models are based on?
   - Recurrent or convolutional neural networks.

2. What configuration do these models use?
   - Encoder-decoder configuration.

3. Which model dispenses with recurrence and convolutions?
   - The Transformer.

4. What mechanism does the Transformer rely on?
   - Attention mechanisms.
